In [3]:
using LinearAlgebra, SparseArrays;
using Latexify;

In [4]:
dummy = Dict{String, Array{String}}("a"=>["b","c"], "b"=>["a"], "c"=>["a","d"], "d"=>["b"])

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [5]:
input=dummy

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [11]:
damping=0.85
function naive_get_matrix(data)
    n=length(dummy)
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, value) in data
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        i=i+1
    end
    M=zeros(Float64, length(dummy), length(dummy))
    for (from, list) in data 
        l=length(list)
        for to in list 
            M[decodeNames[to], decodeNames[from]]=1/l
        end
        M[:,decodeNames[from]].*=damping
        M[:,decodeNames[from]].+=(1-damping)/n
    end
    return (M, encodeNames, decodeNames)
end

naive_get_matrix (generic function with 1 method)

In [12]:
damping=0.85
function sparse_get_matrix(data)
    n=length(dummy)
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, value) in data
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        i=i+1
    end
    M=spzeros(Float64, length(dummy), length(dummy))
    for (from, list) in data 
        l=length(list)
        for to in list 
            M[decodeNames[to], decodeNames[from]]=1/l
        end
        M[:,decodeNames[from]].*=damping
        M[:,decodeNames[from]].+=(1-damping)/n
    end
    return (M, encodeNames, decodeNames)
end

sparse_get_matrix (generic function with 1 method)

In [13]:
M, encode, decode=naive_get_matrix(input)

([0.037500000000000006 0.037500000000000006 0.4625 0.037500000000000006; 0.037500000000000006 0.037500000000000006 0.4625 0.8875; 0.4625 0.8875 0.037500000000000006 0.037500000000000006; 0.4625 0.037500000000000006 0.037500000000000006 0.037500000000000006], ["c", "b", "a", "d"], Dict("c" => 1, "b" => 2, "a" => 3, "d" => 4))

In [14]:
M

4×4 Matrix{Float64}:
 0.0375  0.0375  0.4625  0.0375
 0.0375  0.0375  0.4625  0.8875
 0.4625  0.8875  0.0375  0.0375
 0.4625  0.0375  0.0375  0.0375

In [15]:
latexify(M^2)

L"\begin{equation}
\left[
\begin{array}{cccc}
0.2340625 & 0.4146875 & 0.05343750000000001 & 0.053437500000000006 \\
0.6271875 & 0.4465625 & 0.08531250000000001 & 0.08531250000000001 \\
0.08531250000000001 & 0.08531250000000001 & 0.6271875000000001 & 0.8078125 \\
0.05343750000000001 & 0.053437500000000006 & 0.2340625 & 0.053437500000000006 \\
\end{array}
\right]
\end{equation}
"

In [16]:
function naive(input)
    M, encode, decode = naive_get_matrix(input)
    Λ, X=eigen(M)
    n=size(Λ,1)
    ranks=X[:, n]
    ranks/=sum(ranks)
    sites=map(r-> (encode[r[1]],Float64(r[2])), enumerate(ranks))
    return sort(sites, by= x->x[2], rev=true)
end

naive (generic function with 1 method)

In [17]:
r=naive(input)

4-element Vector{Tuple{String, Float64}}:
 ("a", 0.37824245632609843)
 ("b", 0.30174695606140833)
 ("c", 0.1982530439385918)
 ("d", 0.12175754367390147)

In [18]:
function iterative(input, k)
    M, encode, decode = naive_get_matrix(input)
    n = size(M, 1)
    v = ones(n)/n
    for i in 1:k
        v=M*v
    end
    v=map(r-> (encode[r[1]],Float64(r[2])), enumerate(v))
    return sort(v, by= x->x[2], rev=true)
end

iterative (generic function with 1 method)

In [19]:
v=iterative(input, 40)

4-element Vector{Tuple{String, Float64}}:
 ("a", 0.3782424563260983)
 ("b", 0.3017469560614082)
 ("c", 0.198253043938592)
 ("d", 0.12175754367390175)

In [20]:
function iterative_sparse(input, k)
    M, encode, decode = sparse_get_matrix(input)
    n = size(M, 1)
    v = ones(n)/n
    for i in 1:k
        v=M*v
    end
    v=map(r-> (encode[r[1]],Float64(r[2])), enumerate(v))
    return sort(v, by= x->x[2], rev=true)
end

iterative_sparse (generic function with 1 method)

In [21]:
v=iterative_sparse(input, 40)

4-element Vector{Tuple{String, Float64}}:
 ("a", 0.37824245632609854)
 ("b", 0.3017469560614084)
 ("c", 0.19825304393859214)
 ("d", 0.12175754367390183)

In [22]:
M, encode, decode = sparse_get_matrix(input)

(
 0.0375  0.0375  0.4625  0.0375
 0.0375  0.0375  0.4625  0.8875
 0.4625  0.8875  0.0375  0.0375
 0.4625  0.0375  0.0375  0.0375, ["c", "b", "a", "d"], Dict("c" => 1, "b" => 2, "a" => 3, "d" => 4))